In [3]:
#load libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [4]:
# load data from exploratory_data_analysis notebook for quick logistic regression
%store -r df
df.head()

,patientkey,admissionkey,StartOfCare,DischargeDate,TaskTargetDate,VisitDate,DischargeType,DischargeStatusDescription,TerminalDiagnosis,ICD10CodeShortDesc,...,Depression,Drowsiness,Lack_of_Appetite,Nausea,Pain,Shortness_of_Breath,Tiredness,Wellbeing,cutdate,death_within_7_days
0,2839565,5567524,2017-05-01 00:00:00.000000,2017-07-03,2017-05-01 00:00:00.000,2017-05-01,Death,Expired at Long-term care facility or non skil...,G20,Parkinson's disease,...,0,9,8,0,3,7,9,8,2017-06-26,0
1,2839565,5567524,2017-05-01 00:00:00.000000,2017-07-03,2017-05-15 00:00:00.000,2017-05-15,Death,Expired at Long-term care facility or non skil...,G20,Parkinson's disease,...,11,8,8,0,0,7,8,8,2017-06-26,0
2,2839565,5567524,2017-05-01 00:00:00.000000,2017-07-03,2017-05-29 00:00:00.000,2017-05-29,Death,Expired at Long-term care facility or non skil...,G20,Parkinson's disease,...,11,8,8,0,0,7,9,8,2017-06-26,0
3,2839565,5567524,2017-05-01 00:00:00.000000,2017-07-03,2017-06-12 00:00:00.000,2017-06-12,Death,Expired at Long-term care facility or non skil...,G20,Parkinson's disease,...,11,8,8,0,0,7,8,8,2017-06-26,0
4,2839565,5567524,2017-05-01 00:00:00.000000,2017-07-03,2017-05-04 00:00:00.000,2017-05-04,Death,Expired at Long-term care facility or non skil...,G20,Parkinson's disease,...,0,0,0,0,0,0,11,11,2017-06-26,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 349779 entries, 0 to 349778
Data columns (total 25 columns):
patientkey                    349779 non-null int64
admissionkey                  349779 non-null int64
StartOfCare                   349779 non-null object
DischargeDate                 189408 non-null datetime64[ns]
TaskTargetDate                349779 non-null object
VisitDate                     349779 non-null datetime64[ns]
DischargeType                 190206 non-null object
DischargeStatusDescription    190206 non-null object
TerminalDiagnosis             349733 non-null object
ICD10CodeShortDesc            349733 non-null object
ReferralType                  349778 non-null object
LevelofCare                   348963 non-null object
mortality                     349779 non-null int64
patienttaskkey                349779 non-null int64
Anxiety                       349779 non-null int64
Depression                    349779 non-null int64
Drowsiness                    3

'df' dataframe contains basic identifier columns, as well as a number of datetime objects, pivoted ESAS results, and the outcome of interest: 'death_within_7_days'. What this dataframe does not do (yet) is expand out the ICD10CodeShortDesc column, as this would expand the number of variables from 25 to ~1400. I have done this in another object (df_merge) and if exploratory_data_analysis notebook has been processed, that object is also available for import. Doing so later with a random forest model...<br>
- All the data for training and testing will be from the df_merge dataframe. df is loaded as a reference for which columns to include in smaller logit model before testing random forest on 1400 variable model.
<br>
### Train/Test data splitting
I'll need to split the data into a few objects for data analysis. First, I plan on doing a 70/30% data split for a model building/training data set (what I use to make and tune the models), as well as a test set for a FINAL model evaluation. Within the training set itself, I'll also further split the data into an 80/20% split, with a hold out set used to test the trained models prior to a final validation test. Given the amount of data (~350,000 instances), cross-validation methods could be used in the future, but for a quick project, I'll hold out at this time.

In [6]:
#load dataframe from other notebook
%store -r df_merge 
df_merge.head()

,patientkey,admissionkey,StartOfCare,DischargeDate,TaskTargetDate,VisitDate,DischargeType,DischargeStatusDescription,TerminalDiagnosis,ICD10CodeShortDesc,...,Venous insufficiency (chronic) (peripheral),Ventricular septal defect,Ventricular tachycardia,"Vesical fistula, not elsewhere classified","Viral pneumonia, unspecified",Volunteer activity,Waldenstrom macroglobulinemia,Wernicke's encephalopathy,"X-linked adrenoleukodystrophy, unspecified type",Zoster encephalitis
152342,1605720,5915383,2017-08-02 00:00:00.000000,NaT,2017-08-02 00:00:00.000,2017-08-02,None,None,G35,Multiple sclerosis,...,0,0,0,0,0,0,0,0,0,0
152343,1605720,5915383,2017-08-02 00:00:00.000000,NaT,2017-08-07 00:00:00.000,2017-08-07,None,None,G35,Multiple sclerosis,...,0,0,0,0,0,0,0,0,0,0
152344,1605720,5915383,2017-08-02 00:00:00.000000,NaT,2017-08-09 00:00:00.000,2017-08-09,None,None,G35,Multiple sclerosis,...,0,0,0,0,0,0,0,0,0,0
152345,1605720,5915383,2017-08-02 00:00:00.000000,NaT,2017-08-14 00:00:00.000,2017-08-14,None,None,G35,Multiple sclerosis,...,0,0,0,0,0,0,0,0,0,0
152349,1605720,5915383,2017-08-02 00:00:00.000000,NaT,2017-08-19 00:00:00.000,2017-08-19,None,None,G35,Multiple sclerosis,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y = df_merge['death_within_7_days'] # define the target variable (dependent variable) as y

In [8]:
#drop unnecessary columns
df_merge_dropped = df_merge.drop(['patientkey', 'admissionkey', 'StartOfCare', 'DischargeDate', 'TaskTargetDate',
                                 'VisitDate', 'DischargeType', 'DischargeStatusDescription', 'TerminalDiagnosis',
                                 'ICD10CodeShortDesc', 'ReferralType', 'LevelofCare', 'mortality', 'patienttaskkey',
                                 'cutdate', 'death_within_7_days'], axis=1)
df_merge_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 349779 entries, 152342 to 349778
Columns: 1379 entries, Anxiety to Zoster encephalitis
dtypes: int64(9), uint8(1370)
memory usage: 483.7 MB


In [9]:
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(df_merge_dropped, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(279823, 1379) (279823,)
(69956, 1379) (69956,)


In [10]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279823 entries, 343487 to 100664
Columns: 1379 entries, Anxiety to Zoster encephalitis
dtypes: int64(9), uint8(1370)
memory usage: 386.9 MB


Splitting full dataset (~350,000 rows or instances of the outcome variable) into a Training Set (X) and a Test/Validation set (y). Splitting occurred with a 80/20% split into the respective subsets. Resulting counts: X = 279,823 rows; y = 69,956 rows. Note, at this point, no cross validation is being undertaken for time constraints. Ideally, the Training set would undergo continued subsetting into various validation sets prior to testing models on the test set, but again, this would take extra time. Future work...<br>
<br>
At this point, I have two objects I plan to analyze that includes approximately ~1400 variables. This is too many for a simple logistic regression model, so I will cut out some of the variables to include to better structure the analysis. I'll further remove the variable from X and y sets, but save them as X_logit, y_logit, as I plan on using the full dataset for other model selection methods.

In [11]:
#set columns to keep in dataframes
X_train_logit = X_train[['Anxiety', 'Depression', 'Lack_of_Appetite', 'Nausea', 'Pain', 'Shortness_of_Breath',
                       'Tiredness', 'Wellbeing']]
X_train_logit.head()

,Anxiety,Depression,Lack_of_Appetite,Nausea,Pain,Shortness_of_Breath,Tiredness,Wellbeing
343487,11,11,0,0,0,0,0,11
204008,2,0,4,0,0,0,7,5
232367,0,0,4,0,2,5,5,4
73687,5,0,3,0,0,2,9,5
175101,9,7,5,4,5,9,4,8


In [12]:
X_test_logit = X_test[['Anxiety', 'Depression', 'Lack_of_Appetite', 'Nausea', 'Pain', 'Shortness_of_Breath',
                       'Tiredness', 'Wellbeing']]
X_test_logit.head()

,Anxiety,Depression,Lack_of_Appetite,Nausea,Pain,Shortness_of_Breath,Tiredness,Wellbeing
157685,2,2,2,0,2,1,9,9
45422,4,6,4,0,3,5,5,7
240098,4,11,5,0,0,0,7,8
88734,0,0,8,0,1,0,6,3
201632,2,7,4,0,0,0,7,5


## Logistic Regression models
Now, I'm fitting a simple multivariable logistic regression model to see if the ESAS values alone can predict death within 7 days of the assessment date. Dates aren't included as a variable in the model, but I have filtered the outcome variable in respect to the date of the assessment (i.e. each Did the person pass away within 7 days of the VisitDate where the assessment was completed?)

In [13]:
#fit model
logit = LogisticRegression()

logit_model = logit.fit(X_train_logit, y_train)
predictions = logit.predict(X_test_logit)

In [14]:
#check predictions
predictions[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [15]:
print("Accuracy of logistic regression classifier on test set:", logit_model.score(X_test_logit, y_test))

Accuracy of logistic regression classifier on test set: 0.8588827262850934


Okay, so this model has 86% accuracy. Let's not get carried away though. A model that says 'no' 100% of the time will likely be right quite a bit. 301,165 rows were not within the 7 day window for mortality, and thus 48,614 were. If I set the model to just say everyone isn't going to die within 7 days, I'd be right 86.1% of the time... So this model isn't great.<br>
<br>
let's look at some other performance metrics to identify where the problems lie.

In [16]:
#print confusion matrix
from sklearn.metrics import confusion_matrix
logit_matrix = confusion_matrix(y_test, predictions)
print(logit_matrix)

[[59959   374]
 [ 9498   125]]


So there were 59,909 + 199 = 60,108 correct predictions, but 9,848 errors. What is telling is that there were only 199 correct True Positives, meaning the model identified someone as dying in the next 7 days and that actually happened. In the test set, there were roughly 9500 instances of someone passing away in the next 7 days, so a True Positive count of 199 is pretty terrible. We want to see the lower left quadrant decrease significantly. These cases are known as True negatives, and for the use case of mortality prediction of hospice patients, we want to see this value as low as possible.

In [17]:
#Other performance metrics
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.86      0.99      0.92     60333
          1       0.25      0.01      0.02      9623

avg / total       0.78      0.86      0.80     69956



Some variable descriptions:
- Precision: The ability for the model to not label a sample as positive if it is actually negative. Meaning Finding True negatives (someone isn't dying in the next 7 days, and model agrees). Also referred to as Positive Predictive Value (PPV)
- Recall: The ability of the model to find all true positive events. Someone dies within 7 days, and model agrees. Also referred to as sensitivity.
- F-1: Balance accuracy. This is important, as the data is heavily unbalanced.
- support: just counts of each outcome instance. 1 = death within 7 days.<br>
<br>
    Looking at these stats, I think we can improve on some of these. Precision is a bit low. Recall for '1' events is incredibly low and should be improved with other methods and data.

In [18]:
#Re-balancing model for improvement in performance
#fit model
logit_bal = LogisticRegression(class_weight="balanced")#balancing for outcomes

logit_bal_model = logit_bal.fit(X_train_logit, y_train)
predictions = logit_bal.predict(X_test_logit)

print("Accuracy of logistic regression classifier on test set:", logit_bal_model.score(X_test_logit, y_test))

Accuracy of logistic regression classifier on test set: 0.7747298301789697


On the surface, re-balancing the model results in worse performance, as accuracy was 78% with this model, and 86% with the first logistic regression build. Looking into the errors....

In [19]:
#print confusion matrix
logit_bal_matrix = confusion_matrix(y_test, predictions)
print(logit_bal_matrix)

[[47906 12427]
 [ 3332  6291]]


for our purpose of identifying the individuals at risk for dying in the next 7 days, this does much better. 6,121 were correctly identified as dying, when the outcome was true (True Positives). Additionally, 48,106 were correctly not identified when they didn't. The false positive increased pretty significantly (599 to 12,674 with this model), but in the grand scheme of things, this error might be of less importance (?). The False Positives (model says no death, but they do die) drastically decreased from 9,280 to 3,385 instances. To me, this model is more informative, despite having worse 'accuracy'.

In [20]:
#Other performance metrics
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       0.93      0.79      0.86     60333
          1       0.34      0.65      0.44      9623

avg / total       0.85      0.77      0.80     69956



Other performance statistics also improved.

Assessing variable importance: Odds-ratios


In [21]:
import statsmodels.api as sm
import numpy as np
res = sm.Logit(y_train, X_train_logit)

res.fit().params

C:\tools\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Optimization terminated successfully.
         Current function value: 0.588633
         Iterations 7


Anxiety               -0.013275
Depression             0.007908
Lack_of_Appetite       0.004432
Nausea                 0.019221
Pain                  -0.032590
Shortness_of_Breath   -0.011688
Tiredness             -0.096718
Wellbeing             -0.041947
dtype: float64

In [22]:
result = res.fit()

from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)

print(result.summary())

Optimization terminated successfully.
         Current function value: 0.588633
         Iterations 7
                            Logit Regression Results                           
Dep. Variable:     death_within_7_days   No. Observations:               279823
Model:                           Logit   Df Residuals:                   279815
Method:                            MLE   Df Model:                            7
Date:                 Mon, 12 Feb 2018   Pseudo R-squ.:                 -0.4579
Time:                         15:46:15   Log-Likelihood:            -1.6471e+05
converged:                        True   LL-Null:                   -1.1298e+05
                                         LLR p-value:                     1.000
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Anxiety                -0.0133      0.001    -16.087      0.000      -0.015      -

In [23]:
print(np.exp(result.params))

Anxiety                0.986812
Depression             1.007940
Lack_of_Appetite       1.004442
Nausea                 1.019407
Pain                   0.967935
Shortness_of_Breath    0.988380
Tiredness              0.907812
Wellbeing              0.958920
dtype: float64


In [24]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print(np.exp(conf))

                         2.5%     97.5%        OR
Anxiety              0.985218  0.988410  0.986812
Depression           1.006830  1.009050  1.007940
Lack_of_Appetite     1.003246  1.005640  1.004442
Nausea               1.018094  1.020722  1.019407
Pain                 0.965224  0.970654  0.967935
Shortness_of_Breath  0.986782  0.989981  0.988380
Tiredness            0.905600  0.910029  0.907812
Wellbeing            0.957087  0.960757  0.958920


Most predictive variable was Nausea, followed by depression and lack of appetite. Odds ratios are pretty low, likely to variability within the sample, and size of data. For a more thorough analysis, variables would be added in a step-wise fashion to see how each improves or disproves the model. This is the ideal method, and if given more time, the way I would approach the analysis.

# Tree-based learners (Ensemble Random Forest)

Decision-tree based models are usually great performers and often one of the first true 'machine learning' methods I implement. I love them because they usually perform well, and are still interpretable from a clinical perspective, as the models are built from clinical decision 'rules' (e.g. If age > 65 and weight < 170, do X..). Additionally, tree-based methods also have built-in variable 'importance' indicators, which allows me to include the entire dataset and variables (1,400) in the analysis.<br>
<br>
The remaining model here will be a Random Forest model to hopefully improve on the performance of the existing logistic regression output.

I'll first build a simple decision tree from the logit data (only the ESAS data) for comparison purposes.

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
tree = DecisionTreeClassifier(criterion = 'entropy', random_state = 100, max_depth = 3, min_samples_leaf = 100)
tree.fit(X_train_logit, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=100,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=100, splitter='best')

In [26]:
#accuracy
y_pred_tree = tree.predict(X_test_logit)
print("Accuracy of the decision tree model is: ", accuracy_score(y_test,y_pred_tree))

Accuracy of the decision tree model is:  0.8624421064669221


This model seems to perform pretty well, but concerningly, is similar to the accuracy of just selecting everyone as not dying within 7 days. Looking at the confusion  matrix and performance statistics....

In [27]:
tree_matrix = confusion_matrix(y_test, y_pred_tree)
print(tree_matrix)

[[60333     0]
 [ 9623     0]]


So the one-tree decision tree method did exactly that. It said All people wouldn't die in 7 days, and was right about 86% of time. Looks good, but not helpful clinically.<br>
<br>
Adding multiple decision trees together produces whats called a random forest, which is an ensemble learner (takes the ensemble of multiple trees and combines it into one model).

In [28]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 200)
rf.fit(X_train_logit, y_train)
rf_pred = rf.predict(X_test_logit)
    

In [29]:
print("Accuracy of Random Forest model is: ", accuracy_score(y_test, rf_pred))

Accuracy of Random Forest model is:  0.8968351535250729


Best model yet. The model is right roughly 89.8% of time. Looking into errors and other metrics...

In [30]:
rf_matrix = confusion_matrix(y_test, rf_pred)
print(rf_matrix)

[[58690  1643]
 [ 5574  4049]]


Definitely better than the first tree model. Correctly identified 3,956 people as death in 7 days, and missed 5,523 (model said no, but they actually did...). This is slightly more than the balanced logistic regression model, but in the end, there are way fewer false positives.

In [31]:
print(classification_report(rf_pred, y_test))

             precision    recall  f1-score   support

          0       0.97      0.91      0.94     64264
          1       0.42      0.71      0.53      5692

avg / total       0.93      0.90      0.91     69956



Again, this model produces the optimal outcomes. It balances best the continuum of correctly identifying people, but also minimizes false positives and false negatives. If moving forward and this dataset was all the information we had, I'd recommend this moving forward.

In [32]:
import numpy as np
import 

importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train_logit.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
    
# Plot the feature importances of the forest
plt.figure(figsize=(8,8))
plt.title("Feature importances")
plt.bar(range(x_train.shape[1]), importances[indices],
   color="r", yerr=std[indices], align="center")
feature_names = x_dummies.columns
plt.xticks(range(x_dummies.shape[1]), feature_names)
plt.xticks(rotation=90)
plt.xlim([-1, x_dummies.shape[1]])
plt.show()

Feature ranking:
1. feature 2 (0.240309)
2. feature 6 (0.156870)
3. feature 7 (0.129227)
4. feature 5 (0.124132)
5. feature 0 (0.108364)
6. feature 4 (0.099258)
7. feature 1 (0.089341)
8. feature 3 (0.052499)


<br>
# Random Forest with Full Dataset
Same method, but full dataset. In this one, all the categorical Diagnosis variables have been transposed to a binary outcome, with roughly 1,400 variables.



In [ ]:
rf_full = RandomForestClassifier(n_estimators = 1000) #expanding this to include potentially 1000 different trees. Going for accuracy...
rf_full.fit(X_train, y_train)
rf_full_pred = rf_full.predict(X_test)

In [ ]:
print("Accuracy of Full Random Forest model is: ", accuracy_score(y_test, rf_full_pred))

Random forest using 1000 different trees AND the full dataset with conditions produced the best results to date. Accuracy was 91.1%. NOTE: This model took significantly longer to train (about 4 hours). Other models were done in minutes.

In [ ]:
rf_full_matrix = confusion_matrix(y_test, rf_full_pred)
print(rf_full_matrix)

4,633 out of 5,995 were correctly identified as dying within 7 days. 

In [ ]:
print(classification_report(rf_full_pred, y_test))

Recall for mortality is 77%, which is fantastic. Ideally, we would want precision a bit higher, but this is a great improvement when compared to other models.

In [ ]:
import numpy as np

importances = rf_full.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf_full.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
